### Homogéneisation des lésion par volume décroissant (01 -> 10)

In [ ]:
import SimpleITK as sitk
from pathlib import Path

data_path = Path.cwd().parent.parent / "data"

normalization_folder_path = data_path / "10_masks_with_normalized_lesions_by_descending_volume"
normalization_folder_path.mkdir(parents = True, exist_ok = True)

In [ ]:
"""
01 -> 10
Homogénéiser les lésions par volume décroissant.
"""

mslesseg_path = data_path / "01_MSLesSeg_Dataset"

for mask_path in mslesseg_path.rglob("*_MASK.nii.gz"):
    if len(mask_path.stem.split("_")) == 3:
        arg = mask_path.stem.split("_")[0] + "_" + mask_path.stem.split("_")[1]
        print(arg)
    elif len(mask_path.stem.split("_")) == 2:
        arg = mask_path.stem.split("_")[0]

    # 1) Chemins
    normalization_path = normalization_folder_path / f"{arg}_normalized_labels_mask.nii.gz"  # sortie : labels normalisés

    # 2) Lecture du masque
    mask_img = sitk.ReadImage(mask_path)

    # Sécurité : forcer binaire
    mask_bin = sitk.BinaryThreshold(
        mask_img,
        lowerThreshold=0.5,
        upperThreshold=1e9,
        insideValue=1,
        outsideValue=0
    )
    mask_bin = sitk.Cast(mask_bin, sitk.sitkUInt8)

    # 3) Composantes connexes
    cc_img = sitk.ConnectedComponent(mask_bin)

    # 4) Calcul des volumes
    shape_stats = sitk.LabelShapeStatisticsImageFilter()
    shape_stats.Execute(cc_img)

    labels = list(shape_stats.GetLabels())

    if len(labels) == 0:
        raise RuntimeError("Aucune lésion détectée dans le masque.")

    # Associer label -> volume
    lesions = []
    for lab in labels:
        volume = shape_stats.GetPhysicalSize(lab)  # mm³
        lesions.append((lab, volume))

    # 5) Tri par volume décroissant
    lesions_sorted = sorted(lesions, key=lambda x: x[1], reverse=True)

    print("Ordre des lésions (volume décroissant) :")
    for new_id, (old_lab, vol) in enumerate(lesions_sorted, start=1):
        print(f"Lésion {new_id} ← ancien label {old_lab} | volume = {vol:.1f} mm³")

    # 6) Création de la nouvelle image de labels
    sorted_img = sitk.Image(cc_img.GetSize(), sitk.sitkUInt16)
    sorted_img.CopyInformation(cc_img)

    for new_id, (old_lab, _) in enumerate(lesions_sorted, start=1):
        mask_old = sitk.Equal(cc_img, old_lab)
        sorted_img = sitk.Add(
            sorted_img,
            sitk.Cast(mask_old, sitk.sitkUInt16) * new_id
        )

    # 7) Sauvegarde
    sitk.WriteImage(sorted_img, normalization_path)

    print("\nFichier généré :", normalization_path.name, "\n")


Ordre des lésions (volume décroissant) :
Lésion 1 ← ancien label 4 | volume = 531.0 mm³
Lésion 2 ← ancien label 3 | volume = 482.0 mm³
Lésion 3 ← ancien label 2 | volume = 169.0 mm³
Lésion 4 ← ancien label 1 | volume = 91.0 mm³
Lésion 5 ← ancien label 5 | volume = 59.0 mm³

Fichier généré : P54_normalized_labels_mask.nii.gz 

Ordre des lésions (volume décroissant) :
Lésion 1 ← ancien label 21 | volume = 1246.0 mm³
Lésion 2 ← ancien label 23 | volume = 437.0 mm³
Lésion 3 ← ancien label 18 | volume = 313.0 mm³
Lésion 4 ← ancien label 7 | volume = 292.0 mm³
Lésion 5 ← ancien label 27 | volume = 275.0 mm³
Lésion 6 ← ancien label 26 | volume = 241.0 mm³
Lésion 7 ← ancien label 6 | volume = 227.0 mm³
Lésion 8 ← ancien label 8 | volume = 220.0 mm³
Lésion 9 ← ancien label 1 | volume = 154.0 mm³
Lésion 10 ← ancien label 25 | volume = 138.0 mm³
Lésion 11 ← ancien label 34 | volume = 131.0 mm³
Lésion 12 ← ancien label 15 | volume = 121.0 mm³
Lésion 13 ← ancien label 19 | volume = 116.0 mm³
Lésion